In [1]:
import numpy as np
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk
import pandas as pd
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

def CDG(df, a, b, c):

    # Diccionario de DF por cada estaciÃ³n
    Estaciones = {}
    percent = {}
    # Percentilies deseados
    Prob = [a, b, c]
        
    for columna in df.columns:
        # Crear un DataFrame para cada estaciÃ³n y ordenarlo
        est = df[[columna]].dropna().sort_values(by=columna, ascending=False).reset_index(drop=True)
        est = est.iloc[:-3,:]
        est = est.iloc[1:,:]
        est['Probabilidad'] = 100*(est.index + 1) / (len(est) + 1)
        Estaciones[columna] = est
            
        i = 0
        P = pd.DataFrame(np.zeros((len(Prob), 2)), columns=['Nivel', 'Probabilidad'])

        for p in Prob:
            idx = (est.iloc[:, 1] - p).abs().idxmin()
            P.at[i, 'Nivel'] = float("{:.2f}".format(est.iloc[idx, 0]))
            P.at[i, 'Probabilidad'] = float("{:.2f}".format(est.iloc[idx, 1]))
            i = i + 1
        percent[columna] = P.copy()

    Percentiles.append(percent)
    
    return Percentiles  

def LeerArchivos(a):
    # Crear una ventana raíz
    root = tk.Tk()
    root.withdraw()  # Ocultar la ventana principal
    # Hacer que la ventana esté siempre en el frente
    root.attributes('-topmost', True)
    # Abrir un cuadro de diálogo para seleccionar un archivo CSV
    file_path = filedialog.askopenfilename(
        title=f"Selecciona un archivo CSV de {a}",
        filetypes=(("CSV files", "*.csv"), ("Todos los archivos", "*.*")))
    # Mostrar la ruta del archivo seleccionado
    print(f"Archivo seleccionado: {file_path}")

    # Cerrar la ventana raíz
    root.destroy()
        
    if file_path:  # Verificar si se seleccionó un archivo
        try:
            df = pd.read_csv(file_path, index_col=None)
        except Exception as e:
            print(f"Error al leer el archivo: {e}")
    else:
        print("No se seleccionó ningún archivo.")
        
    df['fecha'] = pd.to_datetime(df['fecha'])
    df = df.set_index(df.columns[0])
                
    #df = df.iloc[:,1:] #EliminarFecahs
        
    return df

def ElegirEstacion(Data):
    
    def Seleccion():
        columna_seleccionada = columna_var.get()
        print(f"Has seleccionado la estación: {columna_seleccionada}")
        root.destroy()  # Cierra la ventana
    
    # Crear la ventana principal de Tkinter
    root = tk.Tk()
    root.title("Selecciona una estación")
    # Hacer que la ventana esté siempre en el frente
    root.attributes('-topmost', True)
    # Variable para almacenar la selección
    columna_var = tk.StringVar()
    columna_var.set(Data.columns[0])  # Valor inicial en el OptionMenu
    dropdown = ttk.OptionMenu(root, columna_var, *Data.columns)
    dropdown.pack(pady=10)
    # Botón para confirmar la selección
    boton = tk.Button(root, text="Seleccionar", command = Seleccion)
    boton.pack(pady=20)
    # Iniciar la ventana
    root.mainloop()
    columna_seleccionada = columna_var.get()
    
    return columna_seleccionada

def ANN():
    # Escalar los datos entre 0 y 1
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled = scaler.fit_transform(pd.DataFrame(DatosPrec[EstP]))
    
    # Dividir los datos en conjunto de entrenamiento y prueba (80% - 20%)
    train_size = int(len(data_scaled) * 0.8)
    train, test = data_scaled[:train_size], data_scaled[train_size:]
    
    # Crear una función para transformar los datos en formato adecuado para LSTM (multivariables)
    def create_dataset(data, look_back=90, forecast_horizon=dias_futuros-1):
        X, y = [], []
        for i in range(len(data) - look_back - forecast_horizon):
            X.append(data[i:(i + look_back), :])  # Tomar todas las características
            y.append(data[(i + look_back):(i + look_back + forecast_horizon), 0])  # Predecir la primera variable
        return np.array(X), np.array(y)
    
    # Usamos 90 días para predecir los próximos 7 días
    look_back = 90
    forecast_horizon = dias_futuros-1
    X_train, y_train = create_dataset(train, look_back, forecast_horizon)
    X_test, y_test = create_dataset(test, look_back, forecast_horizon)
    
    # Reshape para que el modelo LSTM lo entienda (samples, time steps, features)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))
    
    # Parámetros de entrenamiento
    epochs = 20
    batch_size = 5
    
    # Crear el modelo LSTM
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(look_back, 1)))  # Hay 'num_vars' variables de entrada
    model.add(LSTM(50))
    model.add(Dense(forecast_horizon))  # Predecir los próximos 6 días
    
    # Compilar el modelo
    model.compile(loss='mean_absolute_error', optimizer='adam')
    
    # Entrenar el modelo
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), verbose=2)
    
    # Realizar el pronóstico
    test_predict = model.predict(X_test)
    
    # Invertir el escalado para las predicciones y los valores reales
    test_predict_unscaled = []
    y_test_unscaled = []
    
    for i in range(test_predict.shape[0]):
        temp_predict = scaler.inverse_transform(np.concatenate([test_predict[i].reshape(-1, 1), np.tile(X_test[i, -1, 1:], (forecast_horizon, 1))], axis=1))[:, 0]
        temp_y_test = scaler.inverse_transform(np.concatenate([y_test[i].reshape(-1, 1), np.tile(X_test[i, -1, 1:], (forecast_horizon, 1))], axis=1))[:, 0]
        test_predict_unscaled.append(temp_predict)
        y_test_unscaled.append(temp_y_test)
    
    test_predict_unscaled = np.array(test_predict_unscaled)
    y_test_unscaled = np.array(y_test_unscaled)

    return test_predict_unscaled

def LogicaDifusa(P, N):
    
    # Antecedentes (precipitaciónn y nivel) y consecuente (Nivel o caudal)
    PrecP = ctrl.Antecedent([P[EstP].min(), Percentiles[1][EstP].iloc[2, 0], Percentiles[1][EstP].iloc[1, 0], Percentiles[1][EstP].iloc[0, 0], P[EstP].max()], 'Precipitación Pasada')
    PrecP2 = ctrl.Antecedent([P[EstP].min(), Percentiles[1][EstP].iloc[2, 0], Percentiles[1][EstP].iloc[1, 0], Percentiles[1][EstP].iloc[0, 0], P[EstP].max()], 'Precipitación Pasada2')
    NivP = ctrl.Antecedent([N[EstN].min(), Percentiles[0][EstN].iloc[2,0], Percentiles[0][EstN].iloc[1,0], Percentiles[0][EstN].iloc[0, 0], N[EstN].max()], 'Nivel Pasado')
    Niv = ctrl.Consequent([N[EstN].min(), Percentiles[0][EstN].iloc[2,0], Percentiles[0][EstN].iloc[1,0], Percentiles[0][EstN].iloc[0, 0], N[EstN].max()], 'Nivel')
    
    # Definición de membresí­as
    PrecP['Bajo'] = fuzz.trimf(PrecP.universe, [P[EstP].min(), P[EstP].min(), Percentiles[1][EstP].iloc[2, 0]])
    PrecP['Medio-Bajo'] = fuzz.trimf(PrecP.universe, [P[EstP].min(), Percentiles[1][EstP].iloc[2, 0], Percentiles[1][EstP].iloc[1, 0]])
    PrecP['Medio'] = fuzz.trimf(PrecP.universe, [Percentiles[1][EstP].iloc[2, 0], Percentiles[1][EstP].iloc[1, 0], Percentiles[1][EstP].iloc[0, 0]])
    PrecP['Alto'] = fuzz.trapmf(PrecP.universe, [Percentiles[1][EstP].iloc[1, 0], Percentiles[1][EstP].iloc[0, 0], P[EstP].max(), P[EstP].max()])

    PrecP2['Bajo'] = fuzz.trimf(PrecP.universe, [P[EstP].min(), P[EstP].min(), Percentiles[1][EstP].iloc[2, 0]])
    PrecP2['Medio-Bajo'] = fuzz.trimf(PrecP.universe, [P[EstP].min(), Percentiles[1][EstP].iloc[2, 0], Percentiles[1][EstP].iloc[1, 0]])
    PrecP2['Medio'] = fuzz.trimf(PrecP.universe, [Percentiles[1][EstP].iloc[2, 0], Percentiles[1][EstP].iloc[1, 0], Percentiles[1][EstP].iloc[0, 0]])
    PrecP2['Alto'] = fuzz.trapmf(PrecP.universe, [Percentiles[1][EstP].iloc[1, 0], Percentiles[1][EstP].iloc[0, 0], P[EstP].max(), P[EstP].max()])

    NivP['Bajo'] = fuzz.trimf(Niv.universe, [N[EstN].min(), N[EstN].min(), Percentiles[0][EstN].iloc[2, 0]])
    NivP['Medio-Bajo'] = fuzz.trimf(Niv.universe, [N[EstN].min(), Percentiles[0][EstN].iloc[2, 0], Percentiles[0][EstN].iloc[1, 0]])
    NivP['Medio'] = fuzz.trimf(Niv.universe, [Percentiles[0][EstN].iloc[2, 0], Percentiles[0][EstN].iloc[1, 0], Percentiles[0][EstN].iloc[0, 0]])
    NivP['Alto'] = fuzz.trapmf(Niv.universe, [Percentiles[0][EstN].iloc[1, 0], Percentiles[0][EstN].iloc[0, 0], N[EstN].max(), N[EstN].max()])
    
    Niv['Bajo (90-100%)'] = fuzz.trimf(Niv.universe, [N[EstN].min(), N[EstN].min(), Percentiles[0][EstN].iloc[2, 0]])
    Niv['Medio-Bajo (50-90%)'] = fuzz.trimf(Niv.universe, [N[EstN].min(), Percentiles[0][EstN].iloc[2, 0], Percentiles[0][EstN].iloc[1, 0]])
    Niv['Medio (10-50%)'] = fuzz.trimf(Niv.universe, [Percentiles[0][EstN].iloc[2, 0], Percentiles[0][EstN].iloc[1, 0], Percentiles[0][EstN].iloc[0, 0]])
    Niv['Alto (0-10%)'] = fuzz.trapmf(Niv.universe, [Percentiles[0][EstN].iloc[1, 0], Percentiles[0][EstN].iloc[0, 0], N[EstN].max(), N[EstN].max()])

    # REGLAS
    B = ctrl.Rule(PrecP['Bajo'], Niv['Bajo (90-100%)'])
    MB = ctrl.Rule(PrecP['Medio-Bajo'], Niv['Medio-Bajo (50-90%)'])
    M = ctrl.Rule(PrecP['Medio'], Niv['Medio (10-50%)'])
    A = ctrl.Rule(PrecP['Alto'], Niv['Alto (0-10%)'])

    B2 = ctrl.Rule(PrecP2['Bajo'], Niv['Bajo (90-100%)'])
    MB2 = ctrl.Rule(PrecP2['Medio-Bajo'], Niv['Medio-Bajo (50-90%)'])
    M2 = ctrl.Rule(PrecP2['Medio'], Niv['Medio (10-50%)'])
    A2 = ctrl.Rule(PrecP2['Alto'], Niv['Alto (0-10%)'])

    Bn = ctrl.Rule(NivP['Bajo'], Niv['Bajo (90-100%)'])
    MBn = ctrl.Rule(NivP['Medio-Bajo'], Niv['Medio-Bajo (50-90%)'])
    Mn = ctrl.Rule(NivP['Medio'], Niv['Medio (10-50%)'])
    An = ctrl.Rule(NivP['Alto'], Niv['Alto (0-10%)'])
    
    Nivel_ctrl = ctrl.ControlSystem([B, MB, M, A, Bn, MBn, Mn, An, B2, MB2, M2, A2]) #
    Nivel = ctrl.ControlSystemSimulation(Nivel_ctrl)
    
    NR = pd.DataFrame(index=range(dias_pasados), columns=range(dias_futuros))

    for i in range(dias_futuros):
        NR.iloc[:dias_pasados-i,i] = DatosNiv[EstN].iloc[-dias_pasados+i:].values
    
    Pronosticos = pd.DataFrame(np.nan, index=range(dias_pasados), columns=[f"Día {i+1}" for i in range(dias_futuros)], dtype=float)

    resultados = []
    
    Prec_In = pd.DataFrame(index=DatosPrec[EstP].iloc[-dias_pasados:].index, columns=range(dias_futuros))
    Prec_In2 = pd.DataFrame(index=DatosPrec[EstP].iloc[-dias_pasados-1:-1].index, columns=range(dias_futuros))
    
    # Asignar valores de t-1 y t-2 (reales)
    Prec_In[0] = DatosPrec[EstP].iloc[-dias_pasados:].values.flatten()  
    Prec_In2[0] = DatosPrec[EstP].iloc[-dias_pasados-1:-1].values.flatten()
    Prec_In2[1] = DatosPrec[EstP].iloc[-dias_pasados:].values.flatten()
    
    # Asignar predicciones a las columnas restantes
    for i in range(dias_pasados):
        Prec_In.iloc[i, 1:] = test_predict_unscaled[-dias_pasados+i, :]
        Prec_In2.iloc[i, 2:] = test_predict_unscaled[-dias_pasados-1+i, 1:]
    
    DATOS = np.zeros((dias_pasados,4))
    
    for i in range(dias_pasados):
        
        a = dias_pasados - i
        
        Nivel_In = DatosNiv[EstN].iloc[[-a]].to_frame()
        
        for dia in range(dias_futuros):
            
            Nivel.input['Precipitación Pasada'] = Prec_In.iloc[i, dia]
            Nivel.input['Precipitación Pasada2'] = Prec_In2.iloc[i, dia]
            Nivel.input['Nivel Pasado'] = Nivel_In.iloc[dia]
            Nivel.compute()
            Nivel_In.loc[len(Nivel_In)] = float(Nivel.output['Nivel'])
            Pronosticos.iloc[i, dia] = float(Nivel.output['Nivel'])
        
    # Calcular las métricas RMSE, MAE y R²
    metrics = {'RMSE': [], 'MAE': [], 'R²': []}

    for day in range(dias_futuros):

        a = dias_pasados - day
        
        rmse = np.sqrt(mean_squared_error(NR.iloc[:a, day], Pronosticos.iloc[:a, day]))
        mae = mean_absolute_error(NR.iloc[:a, day], Pronosticos.iloc[:a, day])
        r2 = r2_score(NR.iloc[:a, day], Pronosticos.iloc[:a, day])
        metrics['RMSE'].append(rmse)
        metrics['MAE'].append(mae)
        metrics['R²'].append(r2)
        
        print(f"Día {day + 1} - RMSE: {rmse}, MAE: {mae}, R²: {r2}")
        
    
    return NR, Pronosticos, PrecP, Niv, metrics

def confianza2():
    # Definir los percentiles del eje X
    percentiles_x = [2.5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 97.5]
    percentiles_labels = ['P2.5', 'P10', 'P20', 'P30', 'P40', 'P50', 'P60', 'P70', 'P80', 'P90', 'P97.5']

    # Obtener percentiles reales
    real = NR.iloc[-len(Pronosticos):,0]
    percentilesR = np.percentile(real, percentiles_x)

    # Crear DataFrames para cada día y calcular percentiles y rangos en bucle
    dias_percentiles = {}
    for dia in range(1, dias_futuros+1):
        # Ordenar los valores pronosticados
        ordenados = Pronosticos.iloc[:, dia - 1].dropna().sort_values(ascending=True).reset_index(drop=True)

        # Crear un DataFrame para cada día
        dia_df = pd.DataFrame({'Nombre': percentiles_labels})

        # Calcular los percentiles
        dia_df['percentiles'] = [np.percentile(ordenados, i) for i in percentiles_x]

        # Función para calcular los rangos C10, C90, C5, C95
        def calcular_rang(ordenados, percentil, percentiles_x):
            # Obtener el percentil anterior
            if percentil == percentiles_x[0]:
                p_anterior = 0
            else:
                p_anterior = np.percentile(ordenados, percentiles_x[percentiles_x.index(percentil) - 1])

            # Filtrar los datos que están en la marca de clase
            p = ordenados[(ordenados <= np.percentile(ordenados, percentil)) & (ordenados > p_anterior)]

            # Calcular los percentiles
            c10 = np.percentile(p, 10) if len(p) > 0 else np.nan  # Verificar longitud para evitar errores
            c90 = np.percentile(p, 90) if len(p) > 0 else np.nan
            c5 = np.percentile(p, 5) if len(p) > 0 else np.nan
            c95 = np.percentile(p, 95) if len(p) > 0 else np.nan
            return c10, c90, c5, c95

        # Calcular los rangos para cada percentil y agregar al DataFrame
        dia_df[['C10', 'C90', 'C5', 'C95']] = [calcular_rang(ordenados, i, percentiles_x) for i in percentiles_x]

        # Guardar el DataFrame del día en el diccionario
        dias_percentiles[f'Dia_{dia}'] = dia_df

    return dias_percentiles, percentilesR

In [2]:
Datos_Historicos_Nivel = LeerArchivos("datos historicos de nivel")
DatosNiv = LeerArchivos("periodo seleccionado de nivel")
EstN = ElegirEstacion(DatosNiv)
DatosPrec = LeerArchivos("periodo seleccionado de precipitacion")
EstP = ElegirEstacion(DatosPrec)

dias_pasados = int(input("Ingrese el número de días pasados: "))
dias_futuros = int(input("Ingrese el número de días futuros: "))

test_predict_unscaled = ANN()

Archivo seleccionado: C:/Users/juanj/Downloads/Estaciones/Estaciones/H12/Seleccionado/Niveles_H12_relleno.csv
Archivo seleccionado: C:/Users/juanj/Downloads/Estaciones/Estaciones/H12/Seleccionado/Niveles_H12_relleno.csv
Has seleccionado la estación: H12
Archivo seleccionado: C:/Users/juanj/Downloads/Estaciones/Estaciones/H12/NetCDF/ATP01PT02.csv
Has seleccionado la estación: ATP01PT02


Ingrese el número de días pasados:  307
Ingrese el número de días futuros:  7


Epoch 1/20
304/304 - 20s - loss: 0.0590 - val_loss: 0.0534 - 20s/epoch - 65ms/step
Epoch 2/20
304/304 - 13s - loss: 0.0586 - val_loss: 0.0536 - 13s/epoch - 41ms/step
Epoch 3/20
304/304 - 12s - loss: 0.0583 - val_loss: 0.0530 - 12s/epoch - 41ms/step
Epoch 4/20
304/304 - 13s - loss: 0.0581 - val_loss: 0.0514 - 13s/epoch - 41ms/step
Epoch 5/20
304/304 - 12s - loss: 0.0579 - val_loss: 0.0523 - 12s/epoch - 41ms/step
Epoch 6/20
304/304 - 12s - loss: 0.0578 - val_loss: 0.0518 - 12s/epoch - 41ms/step
Epoch 7/20
304/304 - 12s - loss: 0.0577 - val_loss: 0.0510 - 12s/epoch - 41ms/step
Epoch 8/20
304/304 - 12s - loss: 0.0577 - val_loss: 0.0512 - 12s/epoch - 40ms/step
Epoch 9/20
304/304 - 13s - loss: 0.0575 - val_loss: 0.0499 - 13s/epoch - 41ms/step
Epoch 10/20
304/304 - 12s - loss: 0.0576 - val_loss: 0.0516 - 12s/epoch - 41ms/step
Epoch 11/20
304/304 - 12s - loss: 0.0576 - val_loss: 0.0525 - 12s/epoch - 41ms/step
Epoch 12/20
304/304 - 12s - loss: 0.0575 - val_loss: 0.0509 - 12s/epoch - 40ms/step
E

In [3]:
Percentiles = []
Percentiles = CDG(DatosNiv, 10, 50, 90)
Percentiles = CDG(DatosPrec, 0, 80, 90)

NR, Pronosticos, PrecP, Niv, metrics = LogicaDifusa(DatosPrec, DatosNiv)

Día 1 - RMSE: 11.485588443675885, MAE: 9.868328564922706, R²: -3.8405184009109155
Día 2 - RMSE: 19.259018264604748, MAE: 17.9506206162923, R²: -12.573011646227663
Día 3 - RMSE: 23.564223996475288, MAE: 22.87197095312285, R²: -19.286041298433556
Día 4 - RMSE: 24.931170179165427, MAE: 24.382829208174066, R²: -21.678950700648787
Día 5 - RMSE: 25.005699479954625, MAE: 24.45153257118151, R²: -21.80220814257054
Día 6 - RMSE: 24.97358979016705, MAE: 24.418480036006308, R²: -21.725371303324593
Día 7 - RMSE: 24.93275541705994, MAE: 24.376216810990314, R²: -21.62426863035763


In [6]:
dias_percentiles, percentilesR = confianza2()

# Generar PDF

In [769]:
# Generar nombre del archivo PDF dinámico basado en variables y nombres de las columnas
pdf_filename = f'NetCDF_{EstP}_2_Variables_P(t-1)_N(t-1).pdf'

# Crear el archivo PDF para guardar gráficos y métricas
with PdfPages(pdf_filename) as pdf:
    # Página 1: Información general
    plt.figure(figsize=(8, 6))
    plt.text(0.5, 0.9, 'Variables Utilizadas en el Pronóstico:', ha='center', fontsize=14)
    plt.text(0.5, 0.8, f'{EstP}, {EstN}', ha='center', fontsize=12)
    plt.text(0.5, 0.7, f'Variable Pronosticada: Caudal en estación {EstN}', ha='center', fontsize=14)
    plt.text(0.5, 0.6, 'Reglas basadas en precipitación t-1 y nivel t-1', ha='center', fontsize=14)
    plt.text(0.5, 0.5, 'Rangos de: Nivel(10, 50, 90), Precipitación(0, 80, 90)', ha='center', fontsize=12)
    plt.text(0.5, 0.4, 'Métricas de Ajuste:', ha='center', fontsize=14)
    for i in range(dias_futuros):
        plt.text(0.5, 0.3 - i*0.05, f"Día {i + 1} - RMSE: {metrics['RMSE'][i]:.4f}, MAE: {metrics['MAE'][i]:.4f}, R²: {metrics['R²'][i]:.4f}",
                 ha='center', fontsize=12)
    plt.axis('off')
    pdf.savefig()
    plt.close()
    
    # Gráficos de predicción e intervalos de confianza en una fila para cada día
    for day in range(dias_futuros):
        
        a = dias_pasados - day
        Dia = dias_percentiles[f'Dia_{day+1}']
        
        # Crear subplots: 1 fila y 2 columnas
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        # Gráfico 1: Pronóstico vs Realidad
        ax1.set_title(f'Pronóstico del Día {day+1}')
        ax1.plot(DatosPrec.index[-len(NR)+day:], NR.iloc[:a, day], label= 'Nivel Real')
        ax1.plot(DatosPrec.index[-len(NR):], Pronosticos.iloc[:, day], label= 'Pronóstico', color='red')
        ax1.set_xlabel('Fecha')
        ax1.set_ylabel('nivel (m)')
        ax1.legend()
        ax1.grid(True)
    
        # Gráfico 2: Intervalos de confianza
        ax2.set_title(f'Día {day+1} - Intervalo de Confianza')
        ax2.plot(Dia['Nombre'], percentilesR, label='Nivel real', color='black', linewidth=1.5)
    
        # Graficar los intervalos de confianza
        ax2.fill_between(Dia['Nombre'], Dia['C10'], Dia['C90'], color='blue', alpha=0.2, label='C10-C90')
        ax2.fill_between(Dia['Nombre'], Dia['C5'], Dia['C95'], color='gray', alpha=0.2, label='C5-C95')
    
        ax2.set_xlabel('Percentil')
        ax2.set_ylabel('Nivel (m)')
        ax2.legend(loc='upper left')
        ax2.grid(True)
    
        # Ajustar el espaciado entre los subplots
        plt.tight_layout()

        pdf.savefig()
        plt.close()
        
    # Gráfico de todas las métricas por día
    plt.figure(figsize=(10, 5))
    days = range(1, 8)
    plt.plot(days, metrics['RMSE'], label='RMSE')
    plt.plot(days, metrics['MAE'], label='MAE')
    plt.plot(days, metrics['R²'], label='R²')
    plt.title('Métricas de Pronóstico para Cada Día')
    plt.xlabel('Día de Pronóstico')
    plt.ylabel('Métrica')
    plt.legend()
    plt.grid(True)
    pdf.savefig()
    plt.close()
    
    # Gráfico membresías de precipitación y nivel
    plt.figure(figsize=(10, 5))
    Niv.view()
    pdf.savefig()
    plt.close()

    plt.figure(figsize=(10, 5))
    PrecP.view()
    pdf.savefig()
    plt.close()
    
# Mensaje de finalización
print(f"Resultados y gráficos guardados en el archivo {pdf_filename}.")

Resultados y gráficos guardados en el archivo NetCDF_Precipitacion_Ponderada_2_Variables_P(t-1)_N(t-1).pdf.


C:\Users\juanj\.conda\envs\Scikit-fuzzi\lib\site-packages\matplotlib\figure.py:456: UserWarning: matplotlib is currently using a non-GUI backend, so cannot show the figure
  warnings.warn(
C:\Users\juanj\.conda\envs\Scikit-fuzzi\lib\site-packages\matplotlib\figure.py:456: UserWarning: matplotlib is currently using a non-GUI backend, so cannot show the figure
  warnings.warn(


<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>